# 1) 데이터 다운로드

In [1]:
!pip install transformers
!pip install gdown

In [2]:
# 학습 데이터 다운로드
!gdown https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
 # 테스트 데이터 다운로드
!gdown https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT

Downloading...
From (original): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
From (redirected): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6&confirm=t&uuid=b54c0671-a5c9-4240-ae94-fa707ae4cb8d
To: /content/summ_train.json
100% 1.16G/1.16G [00:11<00:00, 105MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT
From (redirected): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT&confirm=t&uuid=e446619d-b7b1-4faa-b738-0482af547b6a
To: /content/summ_test.json
100% 147M/147M [00:01<00:00, 99.4MB/s]


In [3]:
import pandas as pd
DATA_TRAIN_PATH = 'summ_train.json'
train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:4000]
print('학 습 데이터의 개수:', len(train_df))
DATA_TEST_PATH = 'summ_test.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:200]
print('테 스트 데이터의 개수:', len(test_df))

학 습 데이터의 개수: 4000
테 스트 데이터의 개수: 200


In [4]:
#documents 파싱
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)
    outs_df = pd.DataFrame(outs, columns = ['media', 'id', 'article_original', 'abstractive', 'extractive'])
    return outs_df

In [5]:
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

In [6]:
train_data['news'] = train_data['article_original'].apply(lambda x : ' '.join(x))
test_data['news'] = test_data['article_original'].apply(lambda x : ' '.join(x))

# 2) 정수 인코딩



1.   입력, 레이블  정수 인코딩
2.   입력에 'summarize' : 2200, ':' : 153을 맨 앞에 추가
3.   입력과 레이블에 < /s >: 1 을 각각 맨 끝에 추가
4.   레이블에 패딩 토큰 -100을 추가
5.   인코더의 입력이 인코더로, 모델은 디코더의 레이블 예측


# 3) 정수 인코딩을 위한 DATASET 생성


In [7]:
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5TokenizerFast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
import numpy as np
from tqdm import tqdm

In [23]:
class T5SummaryDataset(tf.keras.utils.Sequence):
    def __init__(self, df, tokenizer, max_len, batch_size, ignore_index = -100):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.docs = df
        self.batch_size = batch_size
        self.ignore_index = ignore_index
        self.indices = list(range(len(self.docs)))

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("Index out of range")

        batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch = self.docs.iloc[batch_indices]

        input_ids = []
        labels = []

        for _, instance in batch.iterrows():
            encoded_input = self.tokenizer.encode("summarize:" + instance['news'], max_length = self.max_len, padding = 'max_length', truncation = True)
            input_ids.append(encoded_input)

            encoded_label = self.tokenizer.encode(instance['abstractive'], max_length = self.max_len, padding = 'max_length', truncation = True)
            label = [l if l != self.tokenizer.pad_token_id else self.ignore_index for l in encoded_label]
            labels.append(label)

        #배치 데이터 반환
        return {
            'input_ids' : np.array(input_ids),
            'labels': np.array(labels)
        }

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

# 4) 모델 클래스 선언

In [24]:
model = TFT5ForConditionalGeneration.from_pretrained('paust/pko-t5-base' , from_pt = True)
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')

/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassificati

# 5) 데이터로더 변환

In [41]:
batch_size =8
max_len = 256
lr = 3e-5
max_epochs = 5
warmup_ratio = 0.1

In [43]:
train_dataset = T5SummaryDataset(train_data, tokenizer, max_len = max_len, batch_size = batch_size)
test_dataset = T5SummaryDataset(test_data, tokenizer, max_len = max_len, batch_size = batch_size)

total_steps = len(train_dataset) * max_epochs
warmup_steps = int(total_steps * warmup_ratio)
lr_schedule = CosineDecay(initial_learning_rate = lr, decay_steps = total_steps)
optimizer = Adam(learning_rate = lr_schedule)

In [44]:
len(train_dataset)

500

# 6) 데이터 확인

In [45]:
print("첫번째 샘플의 원문 텍스트:", train_data['news'].loc[0])

첫번째 샘플의 원문 텍스트: ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


In [46]:
print("정수 인코딩 및 패딩 결과:", train_dataset[0]['input_ids'][0])
print(len(train_dataset[0]['input_ids'][0]))

정수 인코딩 및 패딩 결과: [ 7675    78 20359    74 26159    27  7231   480   222  1526   541   222
  7004    17 23431    15    15    15 13238   681   222 18165   222   926
   222  5009  1303   222    84  2667    66  7066    33    73  3415 11261
 11866    15 39816   222  5809  6171   278   222  2533   222 12050   871
   333   222  6778   403   373   222  1745   701   222   863   222  1387
   667   222  2642   429   222  2533   222  1951  2487   354   333   222
  2430   222  1808  4210   222   500    15   222  2533   222  1951  2487
   354   274   222  1650   333   222   527  1963   222  1283   279   222
  1650   222  2233   222  4758 13238   824   222  1967   222   450   222
   804   222 13238   291   222 49511   222  1650  2911   222  1928   222
  2470   466   333   222   336  3502  1116   222  2452   267    15   222
  1387   222  4147   302   222  1283   222   804   222 13238   222  5431
   222  1247  4107   311   222  1404   222    22   348  7231   302   222
   585   222  3738     6   321   22

In [47]:
print("첫번째 샘플의 정수 인코딩 후 복원 결과:", tokenizer.decode(train_dataset[0]['input_ids'][0]))

첫번째 샘플의 정수 인코딩 후 복원 결과: summarize:ha당 조사료 400만원...작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 </s>


In [48]:
print("첫번째 샘플의 요약문:" , train_data['abstractive'].loc[0])

첫번째 샘플의 요약문: 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .


In [49]:
print("첫번째 샘플의 요약문의 정수 인코딩 및 패딩 결과:", train_dataset[0]['labels'][0])

첫번째 샘플의 요약문의 정수 인코딩 및 패딩 결과: [ 5809  6171   278   222  2533   222 12050   871   333   222  6778   403
   373   222  1745   701   222   863   222  1387   667   222  1650   333
   222   527  1963   222  1283   279   222  1650   222  2233   222  4758
 13238   824   222  1967   222   450   222   804   222 13238   291   222
 49511   222  1650  2911   222  1928   222  2470   466   333   222   336
  3502  1116   222     8  2533   222  1951  2487   354     8   333   222
  2430   403   373   222  2642   701   293   222   443   222  1243   222
  3858   349   667   222    19   515   222  3732   349   579   222 14205
   222  2500   284   222 30492  1588   402   389   222  1143  6604   222
    15     1  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -1

In [50]:
test_array = train_dataset[0]['labels'][0]
test_array[test_array == -100] = 0

print("첫번째 샘플의 요약문 레이블:", tokenizer.decode(test_array))

첫번째 샘플의 요약문 레이블: 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


# 7) 어텐션 마스크 함수

In [51]:
def create_attention_mask(input_ids):
    boolean_mask = tf.not_equal(input_ids, tokenizer.pad_token_id)
    return tf.cast(boolean_mask, tf.float32)

# 8) 학습

In [52]:
#훈련 루프
best_loss = float('inf')

#총 훈련할 에폭수 설정
max_epochs = 5
for epoch in range(max_epochs):
    print(f'에포크 {epoch+1}/ {max_epochs}')

    #훈련
    total_train_loss = 0.0
    for batch in tqdm(train_dataset, total = len(train_dataset), desc = '훈련중'):
        attention_mask = create_attention_mask(batch['input_ids'])
        with tf.GradientTape() as tape:
            outputs = model(input_ids = batch['input_ids'],
                            attention_mask = attention_mask,
                            labels = batch['labels'],
                            training = True)
            loss = outputs.loss
        total_train_loss += tf.reduce_mean(loss).numpy() #배치의 평균손실을 누적
        #그레디언트 계산
        gradients =tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    #에폭의 평균 훈련 손실 계산
    avg_train_loss = total_train_loss / len(train_dataset)
    print(f'훈련 손실 : {avg_train_loss: .4f}')

    #평가
    total_val_loss = 0.0
    for batch in tqdm(test_dataset, total = len(test_dataset), desc = '검증 중'):
        attention_mask = create_attention_mask(batch['input_ids'])
        outputs = model(input_ids = batch['input_ids'],
                        attention_mask = attention_mask,
                        labels = batch['labels'],
                        training = False)
        total_val_loss += tf.reduce_mean(outputs.loss).numpy()

    avg_val_loss = total_val_loss / len(test_dataset)
    print(f"검증 손실 :  {avg_val_loss: .4f}")

    #최고 성능 모델 저장
    #현재 에폭의 검증 손실이 이전 최저 손실보다 낮은 경우
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        model.save_pretrained('model')
        print(f"검증 손실이 {best_loss: .4f}로 개선")

    #에포크 종료 시 데이터 셔플
    train_dataset.on_epoch_end()

print('훈련이 완료되었습니다.')


에포크 1/ 5


훈련중: 100%|██████████| 500/500 [21:31<00:00,  2.58s/it]


훈련 손실 :  1.3483


검증 중: 100%|██████████| 25/25 [00:13<00:00,  1.81it/s]


검증 손실 :   1.3247
검증 손실이  1.3247로 개선
에포크 2/ 5


...

검증 손실 :   1.2860
에포크 5/ 5


훈련중: 100%|██████████| 500/500 [21:29<00:00,  2.58s/it]


훈련 손실 :  1.0302


검증 중: 100%|██████████| 25/25 [00:13<00:00,  1.79it/s]

검증 손실 :   1.2867
훈련이 완료되었습니다.


#9) 로드 및 평가

In [53]:
loaded_model = TFT5ForConditionalGeneration.from_pretrained('model')

/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [74]:
def evaluate_model(model, test_datset):
    total_val_loss = 0.0

    for batch in tqdm(test_dataset, total = len(test_dataset), desc = '평가 중'):
        attention_mask = create_attention_mask(batch['input_ids'])
        outputs = model(input_ids = batch['input_ids'],
                        attention_mask = attention_mask,
                        labels = batch['labels'],
                        training = False)
        total_val_loss += tf.reduce_mean(outputs.loss).numpy()
    avg_val_loss = total_val_loss / len(test_dataset)

    return avg_val_loss

In [75]:
test_loss = evaluate_model(loaded_model,  test_dataset)
print(f"테스트 손실: {test_loss:.4f}")

평가 중: 100%|██████████| 25/25 [00:13<00:00,  1.81it/s]

테스트 손실: 1.2842


#  10) 요약문 생성

In [68]:
def summarize(text, model, tokenizer, max_length = 300):
    input_text = 'summarize' + text
    inputs = tokenizer(input_text, return_tensors = "tf", max_length = 512, truncation = True, padding = 'max_length')

    attention_mask = create_attention_mask(inputs['input_ids'])

    summary_ids = model.generate(
        input_ids = inputs['input_ids'],
        attention_mask = attention_mask,
        max_length = max_length,
        num_beams = 7,
        repetition_penalty = 2.0
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens = True)

    return summary

In [69]:
text = test_data.loc[0]['news']
print(text)

[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 "넓고, 체증 없는 '통신 고속도로'가 5G"라며 "대한민국의 대전환이 이제 막 시작됐다"고 기대감을 높였다. 문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 "5G 시대는 우리가 생각하고, 만들면 그것이 세계 표준이 되는 시대"라며 "5G는 대한민국 혁신성장의 인프라"라고 강조했다. 산업화 시대에 고속도로가 우리 경제의 '대동맥' 역할을 했듯, 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이란 설명이다. 문 대통령은 "5G가 각 분야에 융합되면, 정보통신산업을 넘어 자동차, 드론(무인항공기), 로봇, 지능형 폐쇄회로TV(CCTV)를 비롯한 제조업과 벤처에 이르기까지 우리 산업 전체의 혁신을 통한 동반성장이 가능하다"고 밝혔다. 세계 최초 상용화에 성공한 5G가 반도체를 이을 우리 경제의 새 먹거리가 될 것이란 관측이다. 정부는 2026년 세계 5G 시장 규모가 1161조원에 달할 것으로 보고 있다. 작년 반도체 시장 규모가 529조원인 점을 고려하면 2배 이상 큰 미래 시장이 창출되는 셈이다. 문 대통령은 아직은 국민에게 다소 낯선 5G 시대의 미래상을 친절히 설명해 눈길을 끌기도 했다. 문 대통령은 "'지금 스마트폰으로 충분한데, 5G가 왜 필요하지?'라고 생각할 수 있다"며 "4세대 이동통신은 '아직은' 빠르지만 가까운 미래에는 결코 빠르지 않다"고 했다. 그러면서 "자동차가 많아질수록 더 넓은 길이 필요한 것처럼 사물과 사물을 연결하고, 데이터를 주고받는 이동통신망도 더 넓고 빠른 길이 필요하다"고 덧붙였다. 문 대통령은 세계 최초 상용화에 성공한 우리 5G 기술을 널리 알리는 홍보대사를 자처하기도 했다. 5G 시장을 선점하기 위한 각국의 경쟁이 뜨겁게 달아오른 만큼 정부 차원에서 적극 지원하겠다는 방침이다. 문 대통령은 "평창동계올림픽 360도 중계, 작년 4·27 남북한 정상회담 때 프레스센터에서 사용된 스마트월처럼 기회가 생기면 대

In [70]:
summary = summarize(text, model, tokenizer)
print(summary)

문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 "5G는 우리가 생각하고, 만들면 그것이 세계 표준이 되는 시대"라며 "5G가 각 분야에 융합되면, 정보통신산업을 넘어 자동차, 드론, 지능형 폐쇄회로TV(CCTV)를 비롯한 제조업과 벤처에 이르기까지 우리 산업 전체의 혁신을 통한 동반성장이 가능하다"고 말했다.
